In [19]:
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import dataset_collections
import os
import json
import time

In [2]:
server = 'https://usegalaxy.eu/'
api_key = os.environ['my_galaxy_api']
gi = GalaxyInstance(server, key=api_key)

print(gi)

GalaxyInstance object for Galaxy at https://usegalaxy.eu/


In [3]:
gi.histories.get_histories()

[{'model_class': 'History',
  'id': 'ee9cf18ea49d2fb1',
  'name': 'qiime2 test',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/ee9cf18ea49d2fb1',
  'published': False,
  'annotation': None,
  'tags': [],
  'update_time': '2023-01-30T19:11:02.169097'},
 {'model_class': 'History',
  'id': 'ecfc8c06ceae22af',
  'name': 'shotgun paired test run',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/ecfc8c06ceae22af',
  'published': False,
  'annotation': None,
  'tags': [],
  'update_time': '2023-01-24T17:43:51.334962'}]

In [4]:
# gi.histories.show_history('ecfc8c06ceae22af', contents=False)
gi.datasets.show_dataset('4838ba20a6d86765e865eafb64b12ee6')

{'misc_info': 'uploaded fastqsanger.gz file',
 'dataset_id': '4838ba20a6d86765ddf2b5f3ee9cd961',
 'file_size': 4288747228,
 'id': '4838ba20a6d86765e865eafb64b12ee6',
 'uuid': '46e5d9bb-9e6e-45eb-a425-5110a06cc53e',
 'display_apps': [],
 'data_type': 'abc.FastqSangerGz',
 'type': 'file',
 'permissions': {'manage': ['a31b3c57dee55ce4'],
  'access': ['a31b3c57dee55ce4']},
 'peek': '@SRR14143424.1 1/1\nCGTCACTGTGGAAGAAAGAATATTGTTGACGCCTCTCTGAAGCCCTGTCATAACAGAATCCAGCATTTTCTGAGGCTCTATCTCAAGGGAGTTCGCCCACTCGGAAATAGCAGAATCCTGTTTGAA\n+\nAA<AFJJJJJJJJJJJJJJJJFJJJJJFJJJJJ<JJFJJJJ7F7FFFAFJJJJFJFJJF7FJF<AF7AF<FJJJAA<FJJJFJJJJJJJFJ-A<JFJFJJJJJJ<JJJJJJJJJJJJFJJAJFJJJ\n@SRR14143424.2 2/1\n',
 'extension': 'fastqsanger.gz',
 'visible': False,
 'deleted': False,
 'misc_blurb': '4.0 GB',
 'rerunnable': False,
 'sources': [{'model_class': 'DatasetSource',
   'id': 905842,
   'source_uri': 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR141/024/SRR14143424/SRR14143424_1.fastq.gz',
   'extra_files_path': None,
   'tr

In [5]:
# upload file locally
# gi.tools.upload_file('test.txt', 'f3c2b0f3ecac9f02')

# upload file using FTP when it's over 2 G
# gi.tools.upload_from_ftp('test.txt', 'f3c2b0f3ecac9f02')

In [6]:
gi.workflows.get_workflows()

[{'model_class': 'StoredWorkflow',
  'id': 'dde25e07f2db3f7c',
  'name': 'shotgun workflow for a collection of paired inputs',
  'create_time': '2023-01-09T22:04:20.055180',
  'update_time': '2023-01-25T11:58:00.102993',
  'published': False,
  'importable': True,
  'deleted': False,
  'hidden': False,
  'tags': [],
  'latest_workflow_uuid': 'ca9938ac-491c-4bb1-952a-9ed020c63b58',
  'annotations': [''],
  'url': '/api/workflows/dde25e07f2db3f7c',
  'owner': 'yedil_s',
  'source_metadata': None,
  'number_of_steps': 11,
  'show_in_tool_panel': False},
 {'model_class': 'StoredWorkflow',
  'id': '92c6435c8c34b776',
  'name': 'shotgun workflow for a collection of single ended inputs',
  'create_time': '2023-01-22T14:22:15.635209',
  'update_time': '2023-01-25T11:56:03.806581',
  'published': False,
  'importable': True,
  'deleted': False,
  'hidden': False,
  'tags': [],
  'latest_workflow_uuid': '3dd4c22f-2fe6-4903-8067-03ccc969b338',
  'annotations': [''],
  'url': '/api/workflows/92c64

In [7]:
wf_shotgun_paired = gi.workflows.show_workflow('dde25e07f2db3f7c')
wf_shotgun_paired['inputs']

{'0': {'label': 'Input Dataset Collection',
  'value': '',
  'uuid': 'f799a228-043d-4606-8df7-48cf04b37231'}}

In [8]:
def get_project_to_history_id_info():
    with open('projects.json', 'r') as f:
        data = json.load(f)
    return data

def update_project_to_history_id_info(payload):
    with open('projects.json','r+') as f:
        file_data = json.load(f)
        file_data.update(payload)
        f.seek(0)
        json.dump(file_data, f, indent = 4)

def upload_dataset(gi, dataset_links, history_id):
    r = gi.tools.put_url(
        content=dataset_links,
        history_id=history_id,
    )
    print(r)
    return r['outputs']

def shotgun_main(gi, dataset):
    projects = get_project_to_history_id_info()
    for project_id, dataset_link_list in dataset.items():
        
        print(f'creating new history for {project_id}')
        history_id = gi.histories.create_history(project_id)['id']
        print(f'new history with id {history_id} created for {project_id}')

        update_project_to_history_id_info({project_id: history_id})
        
        print(f'uploading datasets to {project_id}')
        dataset_links = '\n'.join(dataset_link_list)
        upload_result = upload_dataset(gi, dataset_links, history_id)
        
        cnt = 1
        while True:
            state_ids = gi.histories.show_history(history_id, contents=False)['state_ids']
            if len(state_ids['ok']) == len(dataset_link_list):
                print(f'uploading datasets to {project_id} successful')
                break
            else:
                print(f'waitting for the dataset to be available {cnt}')
                cnt = cnt + 1
                time.sleep(60)

In [9]:
dataset = {'PRJNA390460': ['ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR589/003/SRR5890763/SRR5890763_1.fastq.gz', 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR589/003/SRR5890763/SRR5890763_2.fastq.gz']}

shotgun_main(gi, dataset)


creating new history for PRJNA390460
new history with id d786d448a802ae4b created for PRJNA390460
uploading datasets to PRJNA390460
{'outputs': [{'id': '4838ba20a6d867655f39fcd3cfc9f5ca', 'hda_ldda': 'hda', 'uuid': 'c2e38a2a-3992-4687-9992-366908276a19', 'hid': 1, 'file_ext': 'auto', 'peek': None, 'model_class': 'HistoryDatasetAssociation', 'name': 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR589/003/SRR5890763/SRR5890763_1.fastq.gz', 'deleted': False, 'purged': False, 'visible': True, 'state': 'queued', 'history_content_type': 'dataset', 'file_size': 0, 'create_time': '2023-01-31T22:25:45.232410', 'update_time': '2023-01-31T22:25:45.249103', 'data_type': 'galaxy.datatypes.data.Data', 'genome_build': '?', 'validated_state': 'unknown', 'validated_state_message': None, 'misc_info': None, 'misc_blurb': None, 'tags': [], 'history_id': 'd786d448a802ae4b', 'metadata_dbkey': '?', 'output_name': 'output0'}, {'id': '4838ba20a6d86765eeb232dc71866fdd', 'hda_ldda': 'hda', 'uuid': '55e1fd88-658a-4ac6-880

KeyboardInterrupt: 

In [20]:
# gi.histories.show_history('d786d448a802ae4b', contents=False)
collection_response = gi.histories.create_dataset_collection(
    history_id='d786d448a802ae4b',
    collection_description=dataset_collections.CollectionDescription(
        name="MyListOfPairedDatasets",
        type="list:paired",
        elements=[
            dataset_collections.CollectionElement(
                name="sample1",
                type="paired",
                elements=[
                    dataset_collections.HistoryDatasetElement(name="forward", id='4838ba20a6d867655f39fcd3cfc9f5ca'),
                    dataset_collections.HistoryDatasetElement(name="reverse", id='4838ba20a6d86765eeb232dc71866fdd'),
                ]
            )
        ]
    )
)